# Managing Model Context in AutoGen AgentChat

## Overview

Effective management of conversation history (model context) is crucial for building efficient and performant AI agents. Sending the entire conversation history to the Language Model (LLM) in every turn can lead to increased costs, slower responses, and hitting token limits.

AutoGen AgentChat provides the `model_context` parameter in `AssistantAgent` to control how conversation history is managed. This notebook explores different strategies for managing model context, including:

- **`UnboundedChatCompletionContext` (Default)**: Sends the full conversation history.
- **`BufferedChatCompletionContext`**: Limits context to the last `n` messages.

## Prerequisites

Ensure you have the necessary packages installed:

In [1]:
!pip install --quiet -U "autogen-agentchat>=0.7" "autogen-ext[openai]>=0.7" rich

## Unbounded Chat Completion Context (Default)

By default, `AssistantAgent` uses `UnboundedChatCompletionContext`, which means the entire conversation history is sent to the model with each new message. This is simple but can be inefficient for long conversations.

In [2]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.ui import Console
from autogen_core.model_context import UnboundedChatCompletionContext

async def run_unbounded_context_example():
    model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
    agent = AssistantAgent(
        name="unbounded_agent",
        model_client=model_client,
        system_message="You are a helpful assistant.",
        model_context=UnboundedChatCompletionContext() # Explicitly set, though it's the default
    )

    print("\n🔄 Conversation with Unbounded Context:")
    print("-" * 40)

    # First turn
    await Console(agent.run_stream(task="What is the capital of France?"))

    # Second turn - model will see previous message
    await Console(agent.run_stream(task="And what about Germany?"))

    await model_client.close()

await run_unbounded_context_example()


🔄 Conversation with Unbounded Context:
----------------------------------------
---------- TextMessage (user) ----------
What is the capital of France?
---------- TextMessage (unbounded_agent) ----------
The capital of France is Paris.
---------- TextMessage (user) ----------
And what about Germany?
---------- TextMessage (unbounded_agent) ----------
The capital of Germany is Berlin.


## Buffered Chat Completion Context

The `BufferedChatCompletionContext` allows you to limit the number of recent messages sent to the LLM. This is useful for controlling context length and reducing costs, especially in long-running conversations where older messages might not be relevant.

In [3]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.ui import Console
from autogen_core.model_context import BufferedChatCompletionContext

async def run_buffered_context_example():
    model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
    agent = AssistantAgent(
        name="buffered_agent",
        model_client=model_client,
        system_message="You are a helpful assistant.",
        model_context=BufferedChatCompletionContext(buffer_size=1) # Only keep the last message
    )

    print("\n🔄 Conversation with Buffered Context (buffer_size=1):")
    print("-" * 40)

    # First turn
    await Console(agent.run_stream(task="What is the capital of France?"))

    # Second turn - model will NOT see the first message, only the system message and the current user message
    await Console(agent.run_stream(task="And what about Germany?"))

    await model_client.close()

await run_buffered_context_example()


🔄 Conversation with Buffered Context (buffer_size=1):
----------------------------------------
---------- TextMessage (user) ----------
What is the capital of France?
---------- TextMessage (buffered_agent) ----------
The capital of France is Paris.
---------- TextMessage (user) ----------
And what about Germany?
---------- TextMessage (buffered_agent) ----------
Could you please provide more context or specify what aspect of Germany you would like to know about? For example, are you interested in its history, culture, economy, politics, travel information, or something else?


## Next Steps

Experiment further with model context management:

1.  **Adjust `buffer_size`**: Observe how different values impact the conversation flow and model responses.
2.  **Longer Conversations**: Test with very long conversations to see the effects of context truncation.
3.  **Cost Optimization**: Consider how these strategies can help reduce API costs for production applications.
4.  **Custom Context Management**: For advanced scenarios, explore creating your own custom `BaseChatCompletionContext` subclasses.